In [1]:
import pandas as pd

In [2]:
train = pd.read_json('../data/poquad_train.json', lines=True).set_index('id')


In [3]:
import sentencepiece as spm

In [13]:
# Load allegro/plt5-large model from Hugging Face
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch

tokenizer = AutoTokenizer.from_pretrained("allegro/plt5-large", use_fast=False)


model = AutoModelWithLMHead.from_pretrained("allegro/plt5-large")

/home/lazydart/miniconda3/envs/tensorflow/lib/python3.11/site-packages/transformers/models/auto/modeling_auto.py:1581: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [27]:
# Generate answers to questions from train["question"] based on train["context"]
def generate_answers(context, question):
    encoded_input = tokenizer(["Pytanie: " + question + " Kontekst: " + context + " </s>"],
                             return_tensors='pt',
                             max_length=512,
                             truncation=True)
    
    output = model.generate(input_ids = encoded_input.input_ids,
                            attention_mask = encoded_input.attention_mask)
    
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    # inputs = tokenizer(context, question, return_tensors="pt")
    # outputs = model(**inputs, decoder_input_ids=inputs["input_ids"])
    
    # logits = outputs.logits
    
    # all_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    # answer_tokens = all_tokens[torch.argmax(logits[0]) : torch.argmax(logits[0]) + 1]
    # answer = tokenizer.decode(outputs)
    
    return output

# Generate answers for all questions in train
answers = train.iloc[:10].apply(lambda x: generate_answers(x["context"], x["question"]), axis=1)



In [34]:
i = 7
print(answers.iloc[i])

print(train.iloc[i].question)

print(train.iloc[i].context)

規. Model PD został szybko zastąpiony przez samoloty Pomilio PE.̋.誰
Jaki model zastąpił samoloty Pomilio PD?
Samoloty Pomilio PD weszły na wyposażenie jednostek bojowych i zarazem do walki od lipca 1917 roku. Pierwsze otrzymały je w tym miesiącu eskadry (Squadriglia) nr 131 i 132, gdzie zastąpiły Pomilio PC. Do działań wchodziły następnie kolejne nowo formowane eskadry Pomilio: 133 (we wrześniu), 134 (w październiku), 135 (w styczniu 1918 roku), 136 (w maju 1918 roku). Eskadra 139, sformowana z dwóch sekcji początkowo przydzielonych do lotnictwa Marynarki (od sierpnia 1917 roku), weszła do działań w styczniu 1918 roku, a w czerwcu odznaczyła się w bitwie nad Piawą, zrzucając 175 bomb. W bitwie tej brały udział też inne eskadry. Model PD został szybko zastąpiony w jednostkach przez PE i skierowany głównie do szkolenia.
